In [4]:
import os
import sys
sys.path.append('../')
from cec2017_py.cec2017.functions import *
from deephive.environment.commons.objective_functions import *
import numpy as np
from deephive.environment.deephive_utils import *
from deephive.environment.utils import *
from deephive.exploration.gp_surrogate import GPSurrogateModule
from deephive.environment.utils import filter_points
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C
from sklearn.gaussian_process.kernels import Matern
from dotenv import load_dotenv
load_dotenv()
import matplotlib.pyplot as plt
from experiments.run_experiment2 import *
from deephive.environment.optimization_functions.test_functions import CEC17
from sko.PSO import PSO

In [5]:
# #Experiment 1: VARIANCE DRIVEN EXPLORERS AND EXPLOITERS
config_path = "../config/exp_config.json"
model_path = "../models/pbest_unfreeze.pth"
model_path_2 = "../models/gbest.pth"

config = parse_config(config_path)

In [6]:
exp_num = 209
iters = 10
exp_name = "exp_" + str(exp_num)
result_path = 'experiments/results/' + exp_name + '/'
os.makedirs(result_path, exist_ok=True)
title = "PSO"

config['freeze'] = True
config['use_gbest'] = True
config['role_std'] = {'explorer': 0.5, 'exploiter': 0.02}
config['variable_std'] = True,
config['action_std'] = 0.3
config['decay_rate'] = 0.99
config['n_agents'] = 10
config['n_dim'] = 2
config['decay_std'] = False
config['decay_start'] = 0
config['split_agents'] = True
config['split_type'] = "use_stds"
config['plot_gbest'] = True
config['use_optimal_value'] = False
config['layer_size'] = [16, 16]
config['iters'] = iters
config['tol'] = 0
config['objective_function'] = "CEC17"
mode = "test"
config["negative"] = True
config["function_id"] = 0
config["use_grid"] = False

config['w'] = 0.7298
config['c1'] = 1.49816
config['c2'] = 1.49816
config['ep_length'] = 20

env, agent_policies = initialize(config, mode=mode, model_path=[model_path, model_path_2])
pbest_policy = agent_policies[0]
gbest_policy = agent_policies[1]
opt_value = env.objective_function.optimal_value(dim=env.n_dim)
run_experiment(env, pbest_policy, config, exp_name)
objective_function = lambda x: -env.objective_function.evaluate(np.array([x]).reshape(1, -1))[0]
lower_bound, upper_bound = env.objective_function.bounds(dim=env.n_dim)[0].tolist(), env.objective_function.bounds(dim=env.n_dim)[1].tolist()

algorithm = PSO(func=objective_function, n_dim=env.n_dim, pop=config['n_agents'], max_iter=config['ep_length'], lb=lower_bound, ub=upper_bound,
                        w = config['w'], c1 = config['c1'], c2 = config['c2'])
run_experiment_other_algorithm(algorithm, env, config, exp_name, title=title)

KeyError: 'std_obs_dim'

In [7]:
def bent_cigar(x: np.ndarray) -> np.ndarray:
    sm = np.sum(x[:, 1:] * x[:, 1:], axis=1)
    sm = sm * 10e6
    return x[:, 0]*x[:, 0] + sm

In [8]:
# generate random parameters
def generate_random_parameters(dim, lower_bound, upper_bound, n):
    return np.random.uniform(lower_bound, upper_bound, (n, dim))

params = generate_random_parameters(2, -100, 100, 5)
print(params)


[[-70.11847003  19.40717191]
 [-93.10818054 -41.85886453]
 [ 89.55232182 -84.37537421]
 [ 20.40002799 -60.19309413]
 [-73.50028815 -91.53619362]]


In [ ]:
function = CEC17(0, negative=False)
opt_param = np.array([[-55.28, -70.43]])
print(function.evaluate(opt_param))
# print(function.evaluate(params))